In [1]:
# -*- coding: UTF-8 -*-
"""
__author__ = "Tereza Novakova"
__credits__ = ["Tereza Novakova"]
__references__=[Jakub Brom]
__license__ = "GPL"
__version__ = "1.0"
__maintainer__ = "Tereza Novakova"
__email__ = "teznovakova@gmail.com"
"""

'\n__author__ = "Tereza Novakova"\n__credits__ = ["Tereza Novakova"]\n__references__=[Jakub Brom]\n__license__ = "GPL"\n__version__ = "1.0"\n__maintainer__ = "Tereza Novakova"\n__email__ = "teznovakova@gmail.com"\n'

In [2]:
#https://automating-gis-processes.github.io/CSC/notebooks/L5/clipping-raster.html

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs
import os
import json
import pprint


In [3]:
############################################################################
############################USER INPUT for fuction##########################
###########################################################################


In [4]:
#input_path = r"C:\Users\Tereza\Documents\UP\DP\SNIMKY\INPUT\SNIMKY_S2_BREZEN_RIJEN"

#Coordinates of bounding box in WGS (Olomouc)
bbox = box(17.1959,49.5578,17.3136,49.6276)

#EPSG code to transfer the data into
epsg_code = 32633

In [10]:
# Read the data
def ClipSatelliteImage (input_tif, output_tif, bbox, epsg_code):
    data = rasterio.open(input_tif)

    #Creating GeoPanda dataframe for bbox
    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))
    
    # Project the Polygon into same CRS as the grid
    geo = geo.to_crs(crs=data.crs.data)
    geo.crs
    
    #pprint.pprint(data.crs)
    #Function to parse features from GeoDataFrame in such a manner that rasterio wants them
    def getFeatures(gdf):
        return [json.loads(gdf.to_json())['features'][0]['geometry']]
    
    #Get Geometry coordinates
    coords = getFeatures(geo)
  
    # Clip the raster with Polygon
    out_img, out_transform = mask(dataset=data, shapes=coords, crop=True)
    
    #Copy metadata and update
    out_meta = data.meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_img.shape[1],
                     "width": out_img.shape[2],
                     "transform": out_transform,
                     "crs": epsg_code}
                    )
    #Save the clipped raster
    with rasterio.open(output_tif, "w", **out_meta) as dest:
            dest.write(out_img)
    return out_img

In [8]:
#CLIP = ClipSatelliteImage(r'C:\Users\Tereza\Documents\UP\DP\SKRIPTY\TEST\NDVI_20200402.TIF', r'C:\Users\Tereza\Documents\UP\DP\SKRIPTY\TEST\CLIPPED.TIF', bbox, 32633)